Handpoints2angles

Notebook that reads the CSV saved Data from for example Dataset DHG and calculates the handangles out of it

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from handpoints2anglesv2 import *
from pathlib import Path
from scipy.spatial.transform import Slerp, Rotation
import os
import re

import importlib
import handpoints2anglesv2
importlib.reload(handpoints2anglesv2)
from handpoints2anglesv2 import *
# import chart_studio
# chart_studio.tools.set_credentials_file(username='mado24', api_key='6yxG1nNuc8NtNNktcJ3D')
# import chart_studio.plotly as plotly

In [2]:
# Namesof the joints that, CSV file does not contain these, named acordingly to the manus gloves

joint_names = {
    0: 'Wrist',
    1: 'Palm',
    2: 'Thumb_CMC',
    3: 'Thumb_MCP',
    4: 'Thumb_IP',
    5: 'Thumb_TIP',
    6: 'Index_MCP',
    7: 'Index_PIP',
    8: 'Index_DIP',
    9: 'Index_TIP',
    10: 'Middle_MCP',
    11: 'Middle_PIP',
    12: 'Middle_DIP',
    13: 'Middle_TIP',
    14: 'Ring_MCP',
    15: 'Ring_PIP',
    16: 'Ring_DIP',
    17: 'Ring_TIP',
    18: 'Pinky_MCP',
    19: 'Pinky_PIP',
    20: 'Pinky_DIP',
    21: 'Pinky_TIP'
}


In [3]:
'''calculates the angles out of the given skeletal data and stores it in an pickle file. Additionally for the stored angles linear interpolation is done. 
The stored pickle with 3D Data points does not undergo interpolation'''

cwd = Path.cwd()
DHG3D_dir = cwd / '..' / 'Data'   / 'DHG2016'
SFINGE_dir = cwd / '..' / 'Data'   / 'DHG2016'

data_dir = SFINGE_dir

DHG_infos = pd.read_csv((data_dir / 'informations_troncage_sequences.txt').resolve(), sep=' ', names=['gesture', 'finger', 'subject', 'essai', 'frame_start', 'frame_end'], decimal=',', dtype=int)
DHG_infos
mean_smallest_timediff = 0.0181546418 #sets the mean timebase for the later interolation


# gesture = 1
# finger = 1
# subject = 1
# essai = 1

# for gesture in os.listdir(data_dir):
#     if os.path.isdir(gesture):
#         for finger in os.listdir((data_dir / gesture).resolve()):
#             if os.path.isdir(finger):
#                 for subject in os.listdir((data_dir / gesture / finger).resolve()):
#                     if os.path.isdir(subject):
#                         for subject in os.listdir((data_dir / gesture / finger).resolve()):
#                     if os.path.isdir(subject):
handgestdata_pts = pd.DataFrame()
handgestdata_angles = pd.DataFrame()

for gesture in os.listdir(data_dir):
    if os.path.isdir(data_dir / gesture) and gesture.startswith('gesture'):
        n_gest = int(re.search(r'\d+', gesture).group())
        for finger in os.listdir((data_dir / gesture).resolve()):
            n_fing = int(re.search(r'\d+', finger).group())
            for subject in os.listdir((data_dir / gesture / finger).resolve()):
                n_subj = int(re.search(r'\d+', subject).group())
                print(str(gesture) + ' ' + str(finger) + ' ' + str(subject))
                for essai in os.listdir((data_dir / gesture / finger / subject).resolve()):
                    n_essai = int(re.search(r'\d+', essai).group())
                    set_dir = (data_dir / gesture / finger / subject / essai).resolve()

                    set_infos = DHG_infos[(DHG_infos['gesture'] == n_gest) & 
                                          (DHG_infos['finger'] == n_fing) & 
                                          (DHG_infos['subject'] == n_subj) & 
                                          (DHG_infos['essai'] == n_essai)].reset_index(drop=True)

                    # Read the CSV file into a DataFrame, roi = region of interest
                    generalinfopath = (set_dir / 'general_information.txt').resolve()
                    datapath = (set_dir / 'skeleton_world.txt').resolve()

                    roi = pd.read_csv(generalinfopath, sep=' ', names=['timestamp', 'roix', 'roiy', 'roiw', 'roih'], decimal=',')

                    #DHG3D = pd.read_csv('U:/profile.V6/Documents/Data/DHG2016/gesture_1/finger_1/subject_1/essai_1/skeleton_world.txt', sep=' ', decimal='.', names=np.arange(0,66), dtype='float64');
                    index = [np.repeat(np.arange(0,22),3), np.tile(['x', 'y', 'z'], 22)]
                    data = np.loadtxt(datapath, delimiter=' ', dtype='float64')
                    single_gest = pd.DataFrame(data, columns = index, dtype=float)
                    # add timestamp from roi frame
                    single_gest.index = roi['timestamp']*1e-7

                    # Data manipulation -> https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
                    single_gest = single_gest.stack(0, future_stack=True).reset_index()
                
                    single_gest.rename(columns={'level_1':'point'}, inplace=True)
                    single_gest['point'] = single_gest['point'].replace(joint_names)

                    single_gest.loc[:,['n_gesture', 'n_finger', 'n_subject', 'n_essai']] = [n_gest, n_fing, n_subj, n_essai]

                    handgestdata_pts = pd.concat([handgestdata_pts, single_gest])

                    # calculate angles
                    angles_list = []
                    for timestamp in np.sort(single_gest['timestamp'].unique()):
                        handpoints = single_gest[single_gest['timestamp'] == timestamp]

                        #angles_list.append(handpoints2angles(handpoints))
                        df = handpoints2angles(handpoints)
                        angles_list.append(df)
                        
                    angles_calc = pd.concat(angles_list).reset_index(drop = True)
                    angles_calc = angles_calc.astype(float)

                    # add label to the according section of the sequence
                    angles_calc.loc[:,'label'] = 0
                    # angles_calc.loc[:,'label'] = angles_calc.loc[:,'label']
                    angles_calc.loc[set_infos.loc[0,'frame_start'] : set_infos.loc[0,'frame_end'], 'label'] = n_gest

                    # interpolate missing values
                    angles_calc['t_diff'] = angles_calc['timestamp'].diff().replace(np.nan, 0).astype(float)

                    angles_calc['n_diffs'] = (angles_calc['t_diff'] / mean_smallest_timediff).round(decimals = 0).astype(int)
                    
                    ts_insert = []
                    for idx in angles_calc[angles_calc['n_diffs'] > 1].index:
                        # calculate timestamps of missing values in between the given timestamps
                        ts_insert.extend(np.linspace(angles_calc.loc[idx-1,'timestamp'], angles_calc.loc[idx,'timestamp'], angles_calc.loc[idx,'n_diffs'], endpoint = False)[1:].tolist())
                    df_ts_insert = pd.DataFrame(ts_insert, columns=['timestamp'])
                    df_ts_insert['interpolated'] = True
                    # megrge calculated timestamps into data and interpolate

                    angles_calc_interp = pd.concat([angles_calc, df_ts_insert], axis = 0)
                    angles_calc_interp.sort_values(by = 'timestamp', inplace = True)
                    angles_calc_interp.set_index('timestamp', inplace = True)
                    
                    pd.set_option('future.no_silent_downcasting', True)
                    angles_calc_interp.loc[:, 'interpolated'] = angles_calc_interp.loc[:, 'interpolated'].fillna(False) # set missing values (original values, not interpolated) to False
                    angles_calc_interp.loc[:, 'interpolated'] = angles_calc_interp.loc[:, 'interpolated'].astype(bool)

                    # interpolate quaternions
                    slerp = Slerp(angles_calc.loc[:, 'timestamp'], Rotation.from_quat(angles_calc.loc[:, ('Handpoint_Quaternion_v1', 'Handpoint_Quaternion_v2', 'Handpoint_Quaternion_v3', 'Handpoint_Quaternion_s')].values))
                    angles_calc_interp.loc[:, ('Handpoint_Quaternion_v1', 'Handpoint_Quaternion_v2', 'Handpoint_Quaternion_v3', 'Handpoint_Quaternion_s')] = slerp(angles_calc_interp.index).as_quat()
                    
                    # interpolate angles + handpoint
                    columns = list(angles_calc_interp.columns)
                    drops = ('interpolated', 'Handpoint_Quaternion_v1', 'Handpoint_Quaternion_v2', 'Handpoint_Quaternion_v3', 'Handpoint_Quaternion_s')
                    columns_to_interpolate = [entry for entry in columns if entry not in drops]
                    angles_calc_interp.loc[:,columns_to_interpolate] = angles_calc_interp.loc[:,columns_to_interpolate].interpolate(method = 'index') # boolean column is not possible to interpolate
                    
                    angles_calc_interp.drop(['n_diffs', 't_diff'], axis = 'columns', inplace = True)
                    angles_calc_interp.reset_index(inplace = True)
                    angles_calc_interp.loc[angles_calc_interp['label'] != n_gest, 'label'] = 0 # set labels of frames betwen gesture '0' and'1' to '0
                    #angles_calc_interp.loc[:, 'label'] = (angles_calc_interp.loc[:, 'label'] / n_gest).round(decimals = 0).astype(int)*n_gest

                    angles_calc_interp.loc[:,['n_gesture', 'n_finger', 'n_subject', 'n_essai']] = [n_gest, n_fing, n_subj, n_essai]
                    handgestdata_angles = pd.concat([angles_calc_interp, handgestdata_angles])


#handgestdata_angles.to_pickle((data_dir / 'handgestdata_angles5_simple.pkl').resolve())
# # version 5 contains updated interpolation of quaternions, which was done wrong in version 4  
# handgestdata_pts.to_pickle((data_dir / 'handgestdata_pts3.pkl').resolve())        

# DHG3D 686 Minuten
# DHG3D 48  min Optimized search, indexing instead of serach in Dataframe
# version 3 contains labels and interpoltation (just of the angles) for equally spaced timestamps

# 144 min
# 43 min

gesture_1 finger_1 subject_1
gesture_1 finger_1 subject_10
gesture_1 finger_1 subject_11
gesture_1 finger_1 subject_12
gesture_1 finger_1 subject_13
gesture_1 finger_1 subject_14
gesture_1 finger_1 subject_15
gesture_1 finger_1 subject_16
gesture_1 finger_1 subject_17
gesture_1 finger_1 subject_18
gesture_1 finger_1 subject_19
gesture_1 finger_1 subject_2
gesture_1 finger_1 subject_20
gesture_1 finger_1 subject_3
gesture_1 finger_1 subject_4
gesture_1 finger_1 subject_5
gesture_1 finger_1 subject_6
gesture_1 finger_1 subject_7
gesture_1 finger_1 subject_8
gesture_1 finger_1 subject_9
gesture_1 finger_2 subject_1
gesture_1 finger_2 subject_10
gesture_1 finger_2 subject_11
gesture_1 finger_2 subject_12
gesture_1 finger_2 subject_13
gesture_1 finger_2 subject_14
gesture_1 finger_2 subject_15
gesture_1 finger_2 subject_16
gesture_1 finger_2 subject_17
gesture_1 finger_2 subject_18
gesture_1 finger_2 subject_19
gesture_1 finger_2 subject_2
gesture_1 finger_2 subject_20
gesture_1 finger_2 su

In [4]:
handgestdata_angles.to_pickle((data_dir / 'handgestdata_angles5.pkl').resolve())
# version 5 contains updated interpolation of quaternions, which was done wrong in version 4  
# handgestdata_pts.to_pickle((data_dir / 'handgestdata_pts3.pkl').resolve())